In [28]:
%matplotlib notebook
#Module to handle regular expressions
import re
#Library for emoji
import emoji
#Import pandas and numpy to handle data
import pandas as pd
import numpy as np

#import libraries for accessing the database
import psycopg2
from sqlalchemy import create_engine
from postgres_credentials import *

#import libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

#import libraries for tokenization and ML
import json;
import keras;
import keras.preprocessing.text as kpt;
from keras.preprocessing.text import Tokenizer;


#Import all libraries for creating a deep neural network
#Sequential is the standard type of neural network with stackable layers
from keras.models import Sequential;
#Dense: Standard layers with every node connected, dropout: avoids overfitting
from keras.layers import Dense, Dropout, Activation;

In [2]:
#Querying the database
def query_database(tabletweets):
    engine = create_engine("postgresql+psycopg2://%s:%s@%s:%d/%s" %(usertwitter, passwordtwitter, hosttwitter, porttwitter, dbnametwitter))
    table = pd.read_sql_query('select * from %s' %tabletweets,con=engine, index_col='id')
    return table

In [30]:
#preprocess text in tweets by removing links, @UserNames, blank spaces, etc.
def preprocessing_text(table):
    #put everythin in lowercase
    table['tweet'] = table['tweet'].str.lower()
    #Replace rt indicating that was a retweet
    table['tweet'] = table['tweet'].str.replace('rt', '')
    #Replace occurences of mentioning @UserNames
    table['tweet'] = table['tweet'].replace(r'@\w+', '', regex=True)
    #Replace links contained in the tweet
    table['tweet'] = table['tweet'].replace(r'http\S+', '', regex=True)
    table['tweet'] = table['tweet'].replace(r'www.[^ ]+', '', regex=True)
    #remove numbers
    table['tweet'] = table['tweet'].replace(r'[0-9]+', '', regex=True)
    #replace special characters and puntuation marks
    table['tweet'] = table['tweet'].replace(r'[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~]', '', regex=True)
    return table    

In [ ]:
def replace_elongated_words():
    #replace repetitions of same character for 1 repetitions
    table['tweet'] = table['tweet'].replace(r'(\w+)(\1){2,}', r'\1', regex=True)
    

In [ ]:
def split_dataset(table):
    
    
    return train_table, test_table
    

In [23]:
#Processing the data: Tokenization
def tokenization_tweets(table):
    tokenization = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True, split=' ')
    tokenization.fit_on_texts(table['tweet'])
    return tokenization

In [24]:
#split dataset
table = query_database('tweets_avengers')
table.head(20)

,created_at,tweet,user_id,retweetcount
id,,,,
1,2018-12-19 01:15:31,RT @BlockB_united: [VIDEO] Lee Sugeun YouTube ...,271496866,0
2,2018-12-19 01:15:35,RT @iamgeekingout: avengers 4 trailer as told ...,2885234679,0
3,2018-12-19 01:15:42,ok so I just watched Avengers: Infinity War fo...,14365631,0
4,2018-12-19 01:15:42,RT @RobertDowneyJr: Always good to know @NASA ...,797228831031508992,0
5,2018-12-19 01:15:44,RT @continentlbkfst: making new oreo products ...,1037117267396501504,0
6,2018-12-19 01:15:45,RT @iamgeekingout: avengers 4 trailer as told ...,2416961083,0
7,2018-12-19 01:15:54,RT @MarkBrooksArt: On the drawing board today....,329815201,0
8,2018-12-19 01:16:01,RT @iamgeekingout: avengers 4 trailer as told ...,1080256784,0
9,2018-12-19 01:16:02,RT @continentlbkfst: making new oreo products ...,245051133,0


In [27]:
table = preprocessing_text(table)
table.head(20)

,created_at,tweet,user_id,retweetcount
id,,,,
1,2018-12-19 01:15:31,video lee sugeun youtube channel subscriber...,271496866,0
2,2018-12-19 01:15:35,avengers trailer as told by spongebob and f...,2885234679,0
3,2018-12-19 01:15:42,ok so i just watched avengers infinity war for...,14365631,0
4,2018-12-19 01:15:42,always good to know has your back,797228831031508992,0
5,2018-12-19 01:15:44,making new oreo products used to be so easy ...,1037117267396501504,0
6,2018-12-19 01:15:45,avengers trailer as told by spongebob and f...,2416961083,0
7,2018-12-19 01:15:54,on the drawing board today scarletwitch aven...,329815201,0
8,2018-12-19 01:16:01,avengers trailer as told by spongebob and f...,1080256784,0
9,2018-12-19 01:16:02,making new oreo products used to be so easy ...,245051133,0


In [ ]:
#Visualizing the data

In [33]:
#Step2: Create a Neural Network

In [5]:
#Create the model
model_nn = Sequential()

In [ ]:
if __name__ == "__main__":
    
    